In [1]:
from cleanData._ftToSingleEvent import singularFT
import pandas as pd
import numpy as np
import sqlite3 as sql
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'ncaa'

In [33]:
df = pd.read_sql("""
                 SELECT * FROM "2017-2018" LIMIT 1000
                 """, sql.connect('ncaa_pbp.db'), index_col='EventID')

In [34]:
df = singularFT(df)

In [35]:
toKeep = df[df['ftCode'] >= 0]['ElapsedSeconds']
df = df[df['ElapsedSeconds'].isin(toKeep)]

In [36]:
df = df[(df['LPoints'] > 0) |
        (df['WPoints'] > 0) |
        (df['ftCode'].isin([0, 1]))]
df = df[~df['ftCode'].isin([2, 3, 4, 5])]
df = df[['Season', 'DayNum', 'WTeamID', 'LTeamID', 'ElapsedSeconds', 'EventType', 'ftCode']]

In [37]:
ft = df[df['ftCode'] >= 0]
shots = df[~df.index.isin(ft.index)]

In [38]:
df

,Season,DayNum,WTeamID,LTeamID,ElapsedSeconds,EventType,ftCode
EventID,,,,,,,
27543280,2018,11,1104,1272,86,made2_jump,NaN
27543298,2018,11,1104,1272,171,miss1_free,0.0
27543300,2018,11,1104,1272,171,made2_lay,NaN
27543351,2018,11,1104,1272,358,miss1_free,0.0
27543375,2018,11,1104,1272,413,miss1_free,0.0
27543377,2018,11,1104,1272,413,made2_jump,NaN
27543382,2018,11,1104,1272,454,made1_free,8.0
27543452,2018,11,1104,1272,761,miss1_free,0.0
27543456,2018,11,1104,1272,777,miss1_free,0.0


In [130]:
df = shots.reset_index().set_index(['WTeamID', 'LTeamID', 'ElapsedSeconds', 'Season', 'DayNum']).\
    join(ft.reset_index().set_index(['WTeamID', 'LTeamID', 'ElapsedSeconds', 'Season', 'DayNum']), rsuffix='_ft')

In [131]:
df.reset_index(inplace=True)
df.set_index('EventID', inplace=True)

In [132]:
df = df[['EventType', 'ftCode_ft', 'EventID_ft', 'EventType_ft']]

In [133]:
df.dropna(inplace=True)

In [134]:
df['EventType'] = df['EventType'] + '_' + df['ftCode_ft'].apply(str)
toDrop = df['EventID_ft']
df.rename(columns={'ftCode_ft': 'ftCode'}, inplace=True)
df = df[['EventType', 'ftCode']]

In [135]:
orig = pd.read_sql("""
                 SELECT * FROM "2017-2018" LIMIT 1000
                 """, sql.connect('ncaa_pbp.db'), index_col='EventID')

In [136]:
orig = orig[~orig.index.isin(toDrop)]

and1 = orig[orig.index.isin(df.index)]
orig = orig[~orig.index.isin(df.index)]

and1['ftCode'] = df['ftCode']
and1['EventType'] = df['EventType']

In [142]:
df = orig.append(and1)
df.sort_index(inplace=True)

In [143]:
df

,DayNum,ElapsedSeconds,EventPlayerID,EventTeamID,EventType,LPoints,LTeamID,Season,WPoints,WTeamID,ftCode,index
EventID,,,,,,,,,,,,
27543273,11,15,648148,1104,assist,0,1272,2018,0,1104,NaN,0
27543274,11,15,648149,1104,made3_jump,0,1272,2018,3,1104,NaN,1
27543275,11,39,650613,1272,turnover,0,1272,2018,0,1104,NaN,2
27543276,11,43,648156,1104,miss2_jump,0,1272,2018,0,1104,NaN,3
27543277,11,43,648148,1104,reb_off,0,1272,2018,0,1104,NaN,4
27543278,11,47,648148,1104,made2_dunk,0,1272,2018,5,1104,NaN,5
27543279,11,76,648146,1104,foul_pers,0,1272,2018,0,1104,NaN,6
27543280,11,86,650607,1272,made2_jump,2,1272,2018,5,1104,NaN,7
27543281,11,103,650620,1272,steal,0,1272,2018,0,1104,NaN,8
